# Creation of a local SQL database with python

**Instructions**

python and SQL are both important skills for data analyst, to manipulating data and extract query

In this project, I will control MySQL remotely through python thanks to mysql.connector, and do following steps: 

- Importing csv file and data cleaning thanks to Pandas (python)
- Connecting Mysql through python and creating SQL database remotely (python)
- Inserting row by row to SQL database (python + SQL) 
- Querying remotely on SQL database, through python (python + SQL)

so excited right? Let's explore some interesting insights ! :)

PS: you can find attached dataframe csv file dog_breed_characteristics.csv, please place it in the same floder of jupyter notebook, to read the file properly 

## Steps:

1. Store the `dog_breed_characteristics.csv` into a pandas DataFrame
2. What are the columns dtypes? For object/strings columns what are the maximal lengths of strings?
2. Create a local SQL database with corresponding columns and columns types, and an autoincrementing primary key
3. Pick up one row of your dataframe, prepare it for inserting it into the database, treating missing values as `NULL`, and test its insertion into the database
4. Insert all rows of the dataframe into the database
5. Make a few check queries to the database 
6. Compute, with SQL queries, the mean of the `Intelligence` column
7. Update your database by inserting into the `Intelligence` column its mean value on each row where it has a `NULL` value.
8. Double check that now your Intelligence column only has non `NULL` values
9. Drop all rows where `Group1` or `Group2` have `NULL` values
10. On your updated database, determine which dogs have highest `AvgPupPrice`

In [2]:
#1 Store the dog_breed_characteristics.csv into a pandas DataFrame

import pandas as pd
dataset = pd.read_csv("dog_breed_characteristics.csv")

dataset.head(5)

,BreedName,AltBreedName,Group1,Group2,MaleWtKg,Temperment,AvgPupPrice,Intelligence,Watchdog,MalaysiaGuardedDog,MalaysiaProhibitedDog,MalaysiaPopularity,PopularityUS2017
0,Affenpinscher,NaN,Toy,Companion,4.0,"Active, Adventurous, Curious, Fun-loving, Play...",1000.0,37.0,1.0,NaN,NaN,NaN,147.0
1,Afghan Hound,NaN,Sporting,Sight,28.0,"Aloof, Clownish, Dignified, Happy, Independent",2250.0,79.0,1.0,NaN,NaN,NaN,93.0
2,African Hunting Dog,NaN,NaN,NaN,23.0,Wild,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Airedale Terrier,Airedale,Terrier,Terrier,26.0,"Alert, Confident, Courageous, Friendly, Intell...",700.0,29.0,4.0,NaN,NaN,NaN,55.0
4,Akbash,NaN,Working,Guardian,53.0,"Affectionate, Alert, Courageous, Independent, ...",600.0,NaN,5.0,NaN,NaN,NaN,NaN


In [3]:
#2 What are the columns dtypes? 

dataset.info()
print("\n")

# For object/strings columns what are the maximal lengths of strings?
string_columns = dataset[["BreedName", "AltBreedName", "Group1","Group2", "Temperment"]]

for column in string_columns: 
    max_len = dataset[column].astype(str).str.len().max() # some of obervations in some columns are NaN, we should convert it into str before count their max length 
    print(f"{column} : {max_len}")
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   BreedName              250 non-null    object 
 1   AltBreedName           35 non-null     object 
 2   Group1                 245 non-null    object 
 3   Group2                 246 non-null    object 
 4   MaleWtKg               249 non-null    float64
 5   Temperment             249 non-null    object 
 6   AvgPupPrice            245 non-null    float64
 7   Intelligence           149 non-null    float64
 8   Watchdog               244 non-null    float64
 9   MalaysiaGuardedDog     6 non-null      float64
 10  MalaysiaProhibitedDog  9 non-null      float64
 11  MalaysiaPopularity     8 non-null      float64
 12  PopularityUS2017       203 non-null    float64
dtypes: float64(8), object(5)
memory usage: 25.5+ KB


BreedName : 45
AltBreedName : 27
Group1 : 12
Group2 : 10
Tem

In [4]:
# 3 Create a local SQL database with corresponding columns and columns types, and an autoincrementing primary key

#1st step connect to the local database
import mysql.connector as mysql

user_name = "root"
pwd = "Youjunjie123!"

db = mysql.connect(
    host="localhost",
    user=user_name,
    passwd=pwd,
    database = "tutorial",
    use_pure=True 
)
cursor = db.cursor()

create_query = """ 
CREATE TABLE IF NOT EXISTS dog_breeds (
id INTEGER PRIMARY KEY AUTO_INCREMENT, 
BreedName TEXT, 
AltBreedName TEXT, 
Group1 TEXT, 
Group2 TEXT, 
MaleWtKg INTEGER, 
Temperment TEXT, 
AvgPupPrice INTEGER, 
Intelligence INTEGER, 
Watchdog INTEGER,
MalaysiaGuardedDog INTEGER,
MalaysiaProhibitedDog INTEGER,
MalaysiaPopularity INTEGER,
PopularityUS2017 INTEGER
)"""

cursor.execute(create_query)

#2nd step to check it out 

cursor.execute("DESCRIBE dog_breeds")
res = cursor.fetchall()
print(res)

[('id', 'int', 'NO', 'PRI', None, 'auto_increment'), ('BreedName', 'text', 'YES', '', None, ''), ('AltBreedName', 'text', 'YES', '', None, ''), ('Group1', 'text', 'YES', '', None, ''), ('Group2', 'text', 'YES', '', None, ''), ('MaleWtKg', 'int', 'YES', '', None, ''), ('Temperment', 'text', 'YES', '', None, ''), ('AvgPupPrice', 'int', 'YES', '', None, ''), ('Intelligence', 'int', 'YES', '', None, ''), ('Watchdog', 'int', 'YES', '', None, ''), ('MalaysiaGuardedDog', 'int', 'YES', '', None, ''), ('MalaysiaProhibitedDog', 'int', 'YES', '', None, ''), ('MalaysiaPopularity', 'int', 'YES', '', None, ''), ('PopularityUS2017', 'int', 'YES', '', None, '')]


In [5]:
# 4 Pick up one row of your dataframe, prepare it for inserting it into the database, treating missing values as NULL, and test its insertion into the database

In [6]:
# clean NaN for the whole table 

# Identify the integer columns
integer_columns = dataset.select_dtypes(include=['float64']).columns

# Fill NaN values with 0 for integer columns
dataset[integer_columns] = dataset[integer_columns].fillna(0)

# Identify the object columns
object_columns = dataset.select_dtypes(include=['object']).columns

# Fill NaN values with "n" for object columns
dataset[object_columns] = dataset[object_columns].fillna("n")

In [7]:
# Here I will pick the 1st row 

row = dataset.iloc[0]

print(row)

print('\n')
# add "" to str column 

row_to_insert = row[:]
indices_str = [0,1,2,3,5]
for i in indices_str:
    row_to_insert[i]="'"+row[i].strip()+"'"   # explain: If you don't add "" to each string column, SQL querry will not recognize it as "String"!!!
    
print(row_to_insert)


insert_query = """INSERT INTO dog_breeds(BreedName, AltBreedName, Group1, Group2, MaleWtKg, Temperment, 
AvgPupPrice, Intelligence, Watchdog, MalaysiaGuardedDog, MalaysiaProhibitedDog, MalaysiaPopularity, PopularityUS2017)
VALUES({},{},{},{},{},{},{},{},{},{},{},{},{} )
"""

query = insert_query.format(*row)

print('\n')
print(query)

cursor.execute(query)


# check insert 
cursor.execute("SELECT * FROM dog_breeds")
cursor.fetchall()

BreedName                                                    Affenpinscher
AltBreedName                                                             n
Group1                                                                 Toy
Group2                                                           Companion
MaleWtKg                                                               4.0
Temperment               Active, Adventurous, Curious, Fun-loving, Play...
AvgPupPrice                                                         1000.0
Intelligence                                                          37.0
Watchdog                                                               1.0
MalaysiaGuardedDog                                                     0.0
MalaysiaProhibitedDog                                                  0.0
MalaysiaPopularity                                                     0.0
PopularityUS2017                                                     147.0
Name: 0, dtype: object




[(901,
  'Affenpinscher',
  'n',
  'Toy',
  'Companion',
  4,
  'Active, Adventurous, Curious, Fun-loving, Playful, Stubborn',
  1000,
  37,
  1,
  0,
  0,
  0,
  147)]

In [8]:
#5 Insert all rows of the dataframe into the database

In [9]:
# refactor these steps of cleaning 
indices_str = [0,1,2,3,5]

def prepare_row(row):
    row_clean = row[:]
    for i in indices_str:
        row_clean[i]="'"+row[i].strip()+"'"
    return row_clean

# drop the previous recorded table 
cursor.execute("DELETE FROM dog_breeds")

# recreate database 
cursor.execute(create_query)

cursor.execute("DESCRIBE dog_breeds")

cursor.fetchall()

[('id', 'int', 'NO', 'PRI', None, 'auto_increment'),
 ('BreedName', 'text', 'YES', '', None, ''),
 ('AltBreedName', 'text', 'YES', '', None, ''),
 ('Group1', 'text', 'YES', '', None, ''),
 ('Group2', 'text', 'YES', '', None, ''),
 ('MaleWtKg', 'int', 'YES', '', None, ''),
 ('Temperment', 'text', 'YES', '', None, ''),
 ('AvgPupPrice', 'int', 'YES', '', None, ''),
 ('Intelligence', 'int', 'YES', '', None, ''),
 ('Watchdog', 'int', 'YES', '', None, ''),
 ('MalaysiaGuardedDog', 'int', 'YES', '', None, ''),
 ('MalaysiaProhibitedDog', 'int', 'YES', '', None, ''),
 ('MalaysiaPopularity', 'int', 'YES', '', None, ''),
 ('PopularityUS2017', 'int', 'YES', '', None, '')]

In [10]:
dataset.iloc[2]

BreedName                African Hunting Dog
AltBreedName                               n
Group1                                     n
Group2                                     n
MaleWtKg                                23.0
Temperment                              Wild
AvgPupPrice                              0.0
Intelligence                             0.0
Watchdog                                 0.0
MalaysiaGuardedDog                       0.0
MalaysiaProhibitedDog                    0.0
MalaysiaPopularity                       0.0
PopularityUS2017                         0.0
Name: 2, dtype: object

In [11]:
# load the whole table into sql database "dog_breeds"

dataset.head()

for index,row in dataset.iterrows():
    SQL_query = insert_query.format(*prepare_row(row))
    cursor.execute(SQL_query)

In [12]:
# 6 Make a few check queries to the database
cursor.execute("SELECT COUNT(*) FROM dog_breeds")
print(f"there are {cursor.fetchall()} rows")
      
print('\n')

cursor.execute("SELECT * FROM dog_breeds LIMIT 10")
print(cursor.fetchall())

there are [(250,)] rows


[(902, 'Affenpinscher', 'n', 'Toy', 'Companion', 4, 'Active, Adventurous, Curious, Fun-loving, Playful, Stubborn', 1000, 37, 1, 0, 0, 0, 147), (903, 'Afghan Hound', 'n', 'Sporting', 'Sight', 28, 'Aloof, Clownish, Dignified, Happy, Independent', 2250, 79, 1, 0, 0, 0, 93), (904, 'African Hunting Dog', 'n', 'n', 'n', 23, 'Wild', 0, 0, 0, 0, 0, 0, 0), (905, 'Airedale Terrier', 'Airedale', 'Terrier', 'Terrier', 26, 'Alert, Confident, Courageous, Friendly, Intelligent, Outgoing', 700, 29, 4, 0, 0, 0, 55), (906, 'Akbash', 'n', 'Working', 'Guardian', 53, 'Affectionate, Alert, Courageous, Independent, Intelligent, Loyal', 600, 0, 5, 0, 0, 0, 0), (907, 'Akita', 'n', 'Working', 'Herding', 44, 'Alert, Courageous, Dignified, Docile, Friendly, Intelligent, Responsive', 1000, 54, 5, 0, 1, 0, 47), (908, 'Alaskan Malamute', 'Malamute', 'Working', 'Northern', 40, 'Affectionate, Devoted, Dignified, Friendly, Loyal, Playful', 1350, 50, 5, 0, 0, 0, 59), (909, 'American Bulldog', '

In [28]:
# 7 Compute, with SQL queries, the mean of the Intelligence column

mean_query = """
SELECT 
AVG(Intelligence)
FROM dog_breeds
WHERE Intelligence != 0 
"""
# Here I filter the Intelligence values which egal to 0 since they were the NaN 
cursor.execute(mean_query)


In [29]:
print(cursor.fetchall())

[(Decimal('38.6443'),)]


In [32]:
# 8 Update your database by inserting into the Intelligence column its mean value on each row where it has a NULL value.

update_query = """UPDATE dog_breeds
SET Intelligence = 38.6443
WHERE Intelligence = 0"""

cursor.execute(update_query)

In [35]:
# 9 Double check that now your Intelligence column only has non NULL values
cursor.execute("SELECT * FROM dog_breeds WHERE Intelligence = 0")   # reminder: I updated all NaN values of numerical columns by 0 at the very begining 
print(cursor.fetchall())

[]


In [37]:
# 10 Drop all rows where Group1 or Group2 have NULL values

drop_query = """
DELETE FROM dog_breeds
WHERE Group1 = 'n' OR Group2 = 'n'   
"""
# reminder: I updated all NaN values of categorical columns by 'n' at the very begining 

cursor.execute(drop_query)


In [39]:
# check out the table again whether it still exists rows, where Group1 or Group2 have NULL values

cursor.execute("SELECT * FROM dog_breeds WHERE Group1 = 'n' OR Group2 = 'n' ")

print(cursor.fetchall())

[]


In [58]:
# 11 On your updated database, determine which dogs have highest AvgPupPrice

highest_query = """
SELECT 
BreedName,
AvgPupPrice
FROM dog_breeds
ORDER BY AvgPupPrice DESC
LIMIT 1
"""
cursor.execute(highest_query)


In [60]:
print(f"dogs have highest AvgPupPrice is {cursor.fetchall()}")

dogs have highest AvgPupPrice is [('Dogo Argentino', 39000)]


In [14]:
#cursor.execute("DELETE FROM dog_breeds")
#cursor.execute("DROP TABLE dog_breeds")